# Predicting Daily Milk Yield - Fall 2025 ML Course Project

In [ ]:
# Felipe Benitez, feb478
# Edwin Torres, EID
# Gora Bepary, EID
# Sankarsh Narayanan, EID

'''
Make cells to explain what dataset is
Objective
Tools used
Summarizing final results, such like "Best local CV RMSE... Best Kaggle RMSE... Kaggle LB Score..., etc." Things to orient reader.
'''

# Data Loading & Initial Inspection

In [ ]:
'''
Goal: show we know what we're working with before touching models.
So show train/test. Other things we could show as example is train and test shape, train head, info, describe, # of rows/columns, target column, presence of categorical columns, missing values, just to name a few. Show whatever is important and useful. This will contribute to Data Exploration + Quality & Clarity.
'''

# Data Cleaning Section 3

# Handling Missing Values 3.1

In [ ]:
'''
Show how we handled missing values here and explain it. Like median, dropping impossible targets like milk yield < 0, etc. 
'''

# Outliers 3.2

In [ ]:
'''
Do what's needed here
'''

# Exploratory Data Analysis 4

# Target Distribution 4.2

# Relationships with Key Features (or things we tried idk) 4.3

# Farm-Level Differences 4.4

In [ ]:
'''
Since we did farm clustering, show why and how, and also how it ended up being wrong in some way. Just talk about why we did this for example
'''

# Feature Engineering 5

In [ ]:
''' 
Here we tell the story of our features
'''

# 5.1 Date Features

In [ ]:
'''
Make a new section for wtv features we tried to make and stuff, show like date, categorical, farm level, other features we tried and how they worked or didnt work
'''

# Baseline Model (Modeling Approach) 6

In [ ]:
'''
Here we show the baseline model we used w/ default ish paramaters. Show it, train it, run it, no optuna, show fold RMSEs, Average CV RMSEs, and explain what it's doing for us and anything else we can add to make this section full
'''

# Modeling Experiments & Improvements 7


In [ ]:
''''
Here show the process we took and not just the final script, subsections could be how we started with Catboost XGBoost and LightGBM (i have how we started in the doc), for each, show how we went on about it, and why we ended up choosing catboost as main model. We did a lot here so no reason to not uti`lize it all. Can make another subsection for hyperparamater tuning and how we used optuna, show the code or like how we integrated it and did it, show the outputs. Also we could show our cross validation strat. Also ensembles, like snapshot, seeds, blending, stacking. 
'''

# Final Model & Leadership Results

In [ ]:
''' 
Short but important, here we show what we stuck with, why, the RMSE, final kaggle RMSE, show a table of the different `models we tried and their results, etc. Write a descr`iption of what we stuck with and why. As always, just make it good and useful for the reader

'''

# Conclusion

In [ ]:
'''
What worked best (CatBoost, features, multi-seed ensemble, blending, alpha). What didnt help (Certain features, some stacking ideas which improved CV but not LB as you can seee in Felipe notebook. What we would do w/ more time)
'''